In [1]:
import pandas as pd
import glob
from os.path import basename

In [2]:
CURSO = 'O57'
MODO_NORMAL = True # Temporal, solo para programación. Se usa cuando las actividades NO están dentro de una carpeta (de Moodle)
RUTA_GENERAL = './INPUT/GRUPOS_PREDEFINITIVAS/'
RUTA_ESPECIFICA = RUTA_GENERAL + CURSO
PREFIJO_CUESTIONARIOS = 'Tarea:'
SUFIJO_ACTIVIDADES = ' (Real)'
NOMENCLATURA_OMITIR = 5
TOLERANCIA_MOODLE = 0.051
NOMENCLATURA_TIENE_PUNTOS = False
# En esta lista solo se ponen cuestionarios. Las TAREAS van en los bloques de actividades.
ACTIVIDADES_CALIFICABLES = [
]
# El último elemento se cambia por True si solo si la actividad es una tarea, puesto que esas NO es posible descargarlas
# y esas toca es verificarlas contra el libro de calificaciones
if MODO_NORMAL:
    BLOQUES_ACTIVIDADES = {
        'Tarea:Sprint 1. Definiendo el Proyecto (Real)' : (['Sprint 1. Definiendo el Proyecto'], 0.1875, True),
        'Tarea:Sprint 2. Diseñando la Base de Datos (Real)' : (['Sprint 2. Diseñando la Base de Datos'], 0.1875, True),
        'Tarea:Sprint 3. Estructurando el Backend (Real)' : (['Sprint 3. Estructurando el Backend'], 0.3125, True),
        'Tarea:Sprint 4. Estructurando el FrontEnd (Real)' : (['Sprint 4. Estructurando el FrontEnd'], 0.3125, True),
    }
else:
    BLOQUES_ACTIVIDADES = {
        'Tarea:Pitch de talento (Real)' : (['Pitch de talento'], 1, True),
    }

In [3]:
def extraerNombre(ruta, punto = False):
    nBase = basename(archivo) 
    tokens = nBase.split('-')
    casiNombre = '-'.join(tokens[NOMENCLATURA_OMITIR:-1])
    nombre = casiNombre.split('.')[0]
    return nombre if not(punto) else casiNombre
def generarNombreColumnaLibro(nombre):
    return PREFIJO_CUESTIONARIOS + nombre + SUFIJO_ACTIVIDADES

def getDfActividadEstudiante(dfs, actividad, codigo):
    dfActividad = dfs[actividad]
    dfActividad = dfActividad[dfActividad['Número de ID'] == codigo]
    return dfActividad
def getNotaActividad(dfActividad, rayita = True):
    columnaCalificacion = dfActividad.columns[8]
    if len(dfActividad) > 0:
        return float(dfActividad[columnaCalificacion].iloc[0])
    return '-' if rayita else 0
def getNotaLibro(notaLibro, rayita = True):
    if notaLibro == '-':
        return notaLibro if rayita else 0.0
    return float(notaLibro)

In [4]:
archivos = glob.glob(RUTA_ESPECIFICA + '/*.csv')
dfs = {}
bandera = False
for archivo in archivos:
    nombre = extraerNombre(archivo, NOMENCLATURA_TIENE_PUNTOS)
    if nombre in ACTIVIDADES_CALIFICABLES:        
        df = pd.read_csv(archivo)
        df = df.drop(len(df) - 1) # Quita el promedio general
        columnaCalificacion = df.columns[8]
        df = df.sort_values(columnaCalificacion)
        test = df[df[columnaCalificacion] == '-']
        if len(test) > 0:
            bandera = True
            print ('\nSE ENCONTRARON ESTUDIANTES SIN ENVIAR LA ACTIVIDAD: ', nombre)
            print(test[['Nombre', 'Apellido(s)', 'Número de ID','Estado']])
        df = df.drop_duplicates('Nombre de usuario', keep = 'last') # Quita todos los intentos y deja el más alto
        dfs[nombre] = df
    else:
        dfs['LIBRO'] = pd.read_csv(archivo)

In [5]:
for indexLibro, rowLibro in dfs['LIBRO'].iterrows():
    if bandera:
        print ('Hay notas sin enviar')
        break
    codEst = rowLibro['Número de ID']
    for actividad in ACTIVIDADES_CALIFICABLES:
        calificacionLibro = getNotaLibro(rowLibro[generarNombreColumnaLibro(actividad)], False)
        dfActividad = getDfActividadEstudiante(dfs, actividad, codEst)
        calificacionAct = getNotaActividad(dfActividad, False)
        if calificacionLibro != calificacionAct and not(calificacionLibro <= calificacionAct <= calificacionLibro + TOLERANCIA_MOODLE):
            print ('=============')
            print ('\nINCONSISTENCIA DETECTADA', codEst, actividad)
            print ('Calificación libro: ', calificacionLibro,'Calificacion actividad: ', calificacionAct)
            #print ((calificacionLibro <= calificacionAct <= calificacionLibro + TOLERANCIA_MOODLE))
            #print (calificacionLibro, calificacionAct, calificacionLibro + TOLERANCIA_MOODLE)
    
    # Verificación de notas en bloques de actividades
    definitivaAcum = 0
    for carpeta, items in BLOQUES_ACTIVIDADES.items():
        notasCarpeta = 0
        for actividad in items[0]:
            if not items[2]:
                dfActividad = getDfActividadEstudiante(dfs, actividad, codEst)
                notasCarpeta += getNotaActividad(dfActividad, rayita = False)
            else:
                notasCarpeta += getNotaLibro(rowLibro[carpeta], False)
        notaCarpetaLibro = getNotaLibro(rowLibro[carpeta], False) 
        notaCarpeta = notasCarpeta/len(items[0])
        definitivaAcum += notaCarpeta*items[1]
        diff = abs(notaCarpeta - notaCarpetaLibro)
        if diff > TOLERANCIA_MOODLE:
            print ('++++++++++++')
            print ('\nINCONSISTENCIA EN CARPETA DETECTADA', codEst, carpeta)
            print ('Calificación libro: ', notaCarpetaLibro,'Calificacion carpeta: ', notaCarpeta)
    definitivaLibro = getNotaLibro(rowLibro['Total del curso (Real)'], False)
    diff = abs(definitivaLibro - round(definitivaAcum, 1))
    if diff > TOLERANCIA_MOODLE*2:
        print ('!-------------!')
        print ('INCONSISTENCIA EN DEFINITIVA GENERADA', codEst)
        print ('Definitiva libro: ', definitivaLibro, 'Definitiva calculada: ', definitivaAcum)